# ICUsICS DB tutorial

ICUsICS is an anonymized database built from the data stored into the Clinical Information System (CIS) database of 6 Intensive Care Units (ICUs) from the Catalan Institute of Health (ICS). Actually, it is a database of databases, because each ICU belongs to a different hospital and each CIS presents its own particularities.

ICUsICS is not hosted as a database 'per se', but as a directory of folders (tables) with parquet files inside (registries). Together with this tutorial there is a .png image (db_map.png) that shows the tables relation and fields names and types. This will be extremely helpful to search, fetch and merge information.

As you can see in the map, there are a total of 13 tables:

d_variables and d_pharma are very small tables. They are dictionaries that contain all the metadata info needed to search the data (names, descriptions, table where data is stored, etc.).

patients, diagnoses, pharma_orders, pharma_records, labresults_numeric, observation_numeric, observation_flagged, observation_categoric, monitored_categoric are medium-size tables, which mean they are splitted by hospital in 6 chunks (the parquet files inside contain the suffix \_h1, \_h2, \_h3, \_h4, \_h5, or \_h6). This will be useful to avoid reading registries of hospitals you don't want to include in your dataset, optimizing queries time.

monitored_numeric and derived_numeric are very large tables, which mean they are splitted by groups of patients in 600 chunks (100 chunks per hospital). The chunks have as suffix the first and last a_patientid in heach chunk (example: 101_109.parquet). This will be useful to avoid reading registries of patiets you don't want to include in your dataset, optimizing queries time.

This tutorial contains demo code to build a dataset using data of hospital 3.

#### First, install fastparquet

In [3]:
pip install fastparquet

  Using cached fastparquet-0.8.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.5 MB)
  Using cached cramjam-2.6.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### Then, import packages

In [4]:
import boto3
import pandas as pd
import numpy as np
import re
from sagemaker import get_execution_role
role = get_execution_role()
s3 = boto3.resource('s3')

### Example: Creating a cohort of patients in hospital 3 with:  
1- ICU_LOS > 2 days  
2- Primary diagnose of pneumonia (any type)   
3- Invasive Mechanical Ventilation (IMV)  
4- APACHE II > 20  
5- Lactate (arterial) > 2mmol/L at first ICU day  
6- Sedative Drugs (VAD)  

### 1- ICU_LOS > 2 days

In [22]:
%%time

patients = pd.read_parquet('s3://icusics-db/patients/patients_h3.parquet')

CPU times: user 40.1 ms, sys: 4.21 ms, total: 44.4 ms
Wall time: 231 ms


In [23]:
patients.head()

,hospital_coded,a_patientid,patientsex,age,height,weight,bmi,hospadmtime,admwardname,distime,diswardname,hospdistime,hospital_outcome
0,3,3000004,M,40,180,120,37,-3,URGÈNCIES,10309,MEDICINA INTERNA,17603,ALIVE
1,3,3000184,F,80,160,80,31,-4505,CIRURGIA GENERAL I DIGESTIVA,3772,CIRURGIA GENERAL I DIGESTIVA,15073,ALIVE
2,3,3000446,F,60,150,70,31,-381,URGÈNCIES,9245,NEFROLOGIA,19099,ALIVE
3,3,3000658,F,60,160,110,43,-1411,GASTROENTEROLOGIA- AP DIGESTIU,29873,ALTRE CENTRE,29873,ALIVE
4,3,3000852,F,40,160,90,35,-461,ANESTESTESIOLOGIA I REANIMACIO,12215,CIRURGIA ORT I TRAUMATOLOGIA,23426,ALIVE


In [15]:
print(patients.shape)
print(patients['a_patientid'].nunique())

(3815, 13)
3815


In [129]:
los2d = patients[patients['distime']>2880].sort_values('a_patientid').reset_index(drop=True) # All times in the database are integers that represent the minuts from icu admission time

In [128]:
print(los2d['a_patientid'].nunique())
print(round(los2d['a_patientid'].nunique()/patients['a_patientid'].nunique()*100,2))

2312
60.6


### 2- Primary diagnose of pneumonia (any type)

In [10]:
diags = pd.read_parquet('s3://icusics-db/diagnoses/diagnoses_h3.parquet')

In [11]:
diags.head()

,a_patientid,diag_type,referencecode,referencecodename
0,3000004,primary,F14.12,Abús de cocaïna amb intoxicació
1,3000004,secondary,F14.1,Abús de cocaïna
2,3000004,secondary,R45.1,Agitació i agitació psicomotora
3,3000004,secondary,D68,Altres tipus de defecte de la coagulació
4,3000004,secondary,D69.59,Altres tipus de trombocitopènia secundària


In [29]:
print(diags.shape)
print(diags['a_patientid'].nunique())

(27774, 4)
3815


In [58]:
pd_pneumo_patlist = tuple(set(diags[(diags['diag_type']=='primary') & (diags['referencecodename'].str.contains('pneum', case=False))]['a_patientid']))

In [130]:
los2d_pneumopd = los2d[los2d['a_patientid'].isin(pd_pneumo_patlist)].sort_values('a_patientid').reset_index(drop=True)

In [131]:
print(los2d_pneumopd['a_patientid'].nunique())
print(round(los2d_pneumopd['a_patientid'].nunique()/patients['a_patientid'].nunique()*100,2))

228
5.98


### 3- Invasive Mechanical Ventilation (IMV)  

#### First, import variables_ref table to look for the variable code

In [61]:
d_variables = pd.read_parquet('s3://icusics-db/d_variables/d_variables.parquet')

In [64]:
d_variables.head()

,hospital_coded,table,a_variableid,choicecode,choicestringvalue,name,abbreviation,description
0,1,derived_numeric,1030000100,NaN,None,Chronic health evaluation,CHE,In APACHE II and A2
1,1,derived_numeric,1030000114,NaN,None,Daily worst APS,APS,Acute physiology score
2,1,derived_numeric,1030000140,NaN,None,Highest 24 h APACHE II,APACHE II,Automatically calculated APACHE II score
3,1,derived_numeric,1030000145,NaN,None,risk (R) of hospital death,R(APACHE II),None
4,1,derived_numeric,1030000160,NaN,None,Major 24 h SAPS II,SAPS II,Càlcul automàtic SAPS II


In [96]:
# define key characters (remember that strings in this db can be in english, catalan or spanish language) to start a blind search

key_chars = 'vent|mec|inv'

imv_result_dummy = d_variables[(d_variables['hospital_coded']==3) & (
    (d_variables['name'].str.contains(key_chars, case=False, na=False)) | (
    d_variables['description'].str.contains(key_chars, case=False, na=False)) | (
    d_variables['choicestringvalue'].str.contains(key_chars, case=False, na=False)))]

print(imv_result_dummy.shape)

print("To many results, so you decide to ask to the mentors and they say that for that hospital, this feature is an observed_categoric \
feature named 'Teràpia real O2' with the choicestringcode 'Vent Mecànica'")

(335, 8)
To many results, so you decide to ask to the mentors and they say that for that hospital, this feature is an observed_categoric feature named 'Teràpia real O2' with the choicestringcode 'Vent Mecànica'


In [97]:
imv_result_dummy.head()

,hospital_coded,table,a_variableid,choicecode,choicestringvalue,name,abbreviation,description
9885,3,derived_numeric,3030001400,NaN,None,LVSW_left ventric stroke work,LVSW,"= SV x ARTmean x 0.0136 ; SV ml, ARTmean mmH..."
9886,3,derived_numeric,3030001410,NaN,None,LVSWi_left vent stroke w index,LVSWi,"= SI x ARTmean x 0.0136 ; SI ml/m², ARTmean ..."
9887,3,derived_numeric,3030001500,NaN,None,RVSW_right ventric stroke work,RVSW,"= SV x PAPmean x 0.0136 ; SV ml, PAPmean mmH..."
9888,3,derived_numeric,3030001510,NaN,None,RVSW_Right vent stroke w index,RVSWi,"= SVI x PAPmean x 0.0136 ; SVI ml/m², PAPmea..."
10461,3,monitored_categoric,3000003812,1.0,IPPV,Evita 2_4 ventilation mode,Evita24 Mode,Draeger Evita 2 dura and Evita 4 ventilation mode


In [98]:
imv_result = imv_result_dummy[(imv_result_dummy['table']=='observed_categoric') & (imv_result_dummy['name'].str.contains('Teràpia real O2', case=False, na=False)) & (
    imv_result_dummy['choicestringvalue'].str.contains('Vent Mecànica', case=False, na=False))]

print('So you finally get your result:')
imv_result

So you finally get your result:


,hospital_coded,table,a_variableid,choicecode,choicestringvalue,name,abbreviation,description
12241,3,observed_categoric,3015002262,12.0,Vent Mecànica,Teràpia real O2,O2 Teràpia,DI 21.CONTROL RESPIRATORI.\nVariable utilitzad...


#### Get patients with IMV registries for those who have accomplished with the inclusion criteria up to now

In [80]:
%%time

moncat = pd.read_parquet('s3://icusics-db/observed_categoric/observed_categoric_h3.parquet')

CPU times: user 399 ms, sys: 327 ms, total: 726 ms
Wall time: 1.44 s


In [81]:
moncat.head()

,a_patientid,a_variableid,time,choicecode
0,3000004,3010000100,1019,6.0
1,3000004,3010000100,2879,6.0
2,3000004,3010000100,2999,6.0
3,3000004,3010000100,3079,6.0
4,3000004,3010000100,3140,6.0


In [86]:
%%time

imv_patlist = tuple(set(moncat[(moncat['a_variableid']==3015002262) & (moncat['choicecode']==12)]['a_patientid']))

CPU times: user 45.7 ms, sys: 472 µs, total: 46.2 ms
Wall time: 40.3 ms


In [132]:
los2d_pneumopd_imv = los2d_pneumopd[los2d_pneumopd['a_patientid'].isin(imv_patlist)].sort_values('a_patientid').reset_index(drop=True)

In [133]:
print(los2d_pneumopd_imv['a_patientid'].nunique())
print(round(los2d_pneumopd_imv['a_patientid'].nunique()/patients['a_patientid'].nunique()*100,2))

131
3.43


### 4- APACHE II > 20 

In [99]:
# define key characters (remember that strings in this db can be in english, catalan or spanish language) to start a blind search

key_chars = 'apache'

apache_result_dummy = d_variables[(d_variables['hospital_coded']==3) & (
    (d_variables['name'].str.contains(key_chars, case=False, na=False)) | (
    d_variables['description'].str.contains(key_chars, case=False, na=False)) | (
    d_variables['choicestringvalue'].str.contains(key_chars, case=False, na=False)))]

print(apache_result_dummy.shape)

print("To many results, so you decide to ask to the mentors and they say that for that hospital, this feature is a derived_numeric \
feature named 'APACHE 2 validado'")

(60, 8)
To many results, so you decide to ask to the mentors and they say that for that hospital, this feature is a derived_numeric feature named 'APACHE 2 validado'


In [100]:
apache_result = apache_result_dummy[(apache_result_dummy['table']=='derived_numeric') & (apache_result_dummy['name'].str.contains('apache 2 validado', case=False, na=False))]

print('So you finally get your result:')
apache_result

So you finally get your result:


,hospital_coded,table,a_variableid,choicecode,choicestringvalue,name,abbreviation,description
9870,3,derived_numeric,3030000350,NaN,None,APACHE 2 validado,APACHE 2 man,Validated APACHE II score


#### Get patients with an APACHE II higher of 20 for those who have accomplished with the inclusion criteria up to now

In [141]:
%%time

bucket = s3.Bucket('icusics-db')
apacheII_20_patlist = tuple()

for my_bucket_object in bucket.objects.all():
    
    if all(x in my_bucket_object.key for x in ['derived_numeric_']):
        
        boundaries = re.findall('\d+', my_bucket_object.key)
        ub = boundaries.pop(1)
        lb = boundaries.pop()
        for patid in sorted(list(set(los2d_pneumopd_imv['a_patientid'].astype(str)))):
            if patid>lb and patid<ub:
                chunk = pd.read_parquet(f's3://icusics-db/{my_bucket_object.key}')
                apacheII_20_patlist_chunk = tuple(set(chunk[(chunk['a_variableid']==3030000350) & (chunk['value']>20) & (
                    chunk['a_patientid'].isin(tuple(set(los2d_pneumopd_imv['a_patientid']))))]['a_patientid']))
                apacheII_20_patlist = apacheII_20_patlist + apacheII_20_patlist_chunk
                
apacheII_20_patlist_unique = tuple(set(apacheII_20_patlist))

CPU times: user 14 s, sys: 5.77 s, total: 19.8 s
Wall time: 35.7 s


In [142]:
len(apacheII_20_patlist_unique)

60

In [150]:
los2d_pneumopd_imv_apacheII20 = los2d_pneumopd_imv[los2d_pneumopd_imv['a_patientid'].isin(apacheII_20_patlist_unique)].sort_values('a_patientid').reset_index(drop=True)

In [151]:
print(los2d_pneumopd_imv_apacheII20['a_patientid'].nunique())
print(round(los2d_pneumopd_imv_apacheII20['a_patientid'].nunique()/patients['a_patientid'].nunique()*100,2))

60
1.57


### 5- Lactate (arterial) > 2mmol/L at first ICU day 

In [146]:
# define key characters (remember that strings in this db can be in english, catalan or spanish language) to start a blind search

key_chars = 'lactat'

lactate_result_dummy = d_variables[(d_variables['hospital_coded']==3) & (
    (d_variables['name'].str.contains(key_chars, case=False, na=False)) | (
    d_variables['description'].str.contains(key_chars, case=False, na=False)) | (
    d_variables['choicestringvalue'].str.contains(key_chars, case=False, na=False)))]

print(lactate_result_dummy.shape)

print("To many results, so you decide to ask to the mentors and they say that for that hospital, this feature is a labresult_numeric \
feature that contains 'GSA' in its abbreviation")

(7, 8)
To many results, so you decide to ask to the mentors and they say that for that hospital, this feature is a labresult_numeric feature that contains 'GSA' in its abbreviation


In [147]:
lactate_result = lactate_result_dummy[(lactate_result_dummy['table']=='labresults_numeric') & (lactate_result_dummy['name'].str.contains('lactat', case=False, na=False)) & (
    lactate_result_dummy['abbreviation'].str.contains('GSA', case=False, na=False))]

print('So you finally get your result:')
lactate_result

So you finally get your result:


,hospital_coded,table,a_variableid,choicecode,choicestringvalue,name,abbreviation,description
10105,3,labresults_numeric,3024000658,NaN,None,Lactat art GSA,Lactat a GSA,None
10141,3,labresults_numeric,3024000704,NaN,None,aSan-Lactat,Lactat _GSA,None


#### Get patients with an arterial lactate higher than 2mmol/L during the first ICU day for those who have accomplished with the inclusion criteria

In [148]:
%%time

labres = pd.read_parquet('s3://icusics-db/labresults_numeric/labresults_numeric_h3.parquet')

CPU times: user 97.8 ms, sys: 71.3 ms, total: 169 ms
Wall time: 372 ms


In [149]:
labres.head()

,a_patientid,a_variableid,time,value
0,3000446,3020000100,-13,30.2
1,3000446,3020000100,74,31.2
2,3000446,3020000100,1642,27.5
3,3000446,3020000100,3312,31.1
4,3000446,3020000100,4343,28.2


In [157]:
%%time

lac2_fd_patlist = tuple(set(labres[(labres['a_variableid'].isin([3024000658,3024000704])) & (labres['value']>2) & (labres['time']<1440) & (
    labres['a_patientid'].isin(tuple(set(los2d_pneumopd_imv_apacheII20['a_patientid']))))]['a_patientid']))

CPU times: user 21.8 ms, sys: 0 ns, total: 21.8 ms
Wall time: 19.6 ms


In [158]:
len(lac2_fd_patlist)

26

In [173]:
los2d_pneumopd_imv_apacheII20_lac2fd = los2d_pneumopd_imv_apacheII20[los2d_pneumopd_imv_apacheII20['a_patientid'].isin(lac2_fd_patlist)].sort_values(
    'a_patientid').reset_index(drop=True)

In [174]:
print(los2d_pneumopd_imv_apacheII20_lac2fd['a_patientid'].nunique())
print(round(los2d_pneumopd_imv_apacheII20_lac2fd['a_patientid'].nunique()/patients['a_patientid'].nunique()*100,2))

26
0.68


### 6- Sedative Drugs (VAD)

In [162]:
d_pharma = pd.read_parquet(f's3://icusics-db/d_pharma/d_pharma.parquet')

In [163]:
d_pharma.head()

,hospital_coded,a_pharmaid,pharmaname,pharmagroupname,pharmaformunit,pharmadoseunit,pharmadoseformratio,pharmavolumeunit,pharmavolumeformratio
0,1,1001000255,ABACAVIR 300 MG COMP,None,comp,mg,300.0,ml,0.0
1,1,1001000256,AIGUA,Nutrició Enteral,ml,ml,1.0,ml,1.0
2,1,1001000258,BICARBONAT SODIC 1/6M,Serumteràpia,ml,ml,1.0,ml,1.0
3,1,1001000259,BICARBONAT SODIC 1M,Serumteràpia,ml,ml,1.0,ml,1.0
4,1,1001000275,GELATINA 3%,Serumteràpia,ml,ml,1.0,ml,1.0


In [168]:
# define key characters (remember that strings in this db can be in english, catalan or spanish language) to start a blind search

key_chars = 'sed'

sed_results_dummy = d_pharma[(d_pharma['hospital_coded']==3) & (d_pharma['pharmagroupname'].str.contains(key_chars, case=False, na=False))]

print(sed_results_dummy.shape)

print("We have 39 drugs in the group of sedatives, so we save all them in a tuple and search for them in drugs table'")

sedatives_ids = tuple(set(sed_results_dummy['a_pharmaid']))

(39, 9)
We have 39 drugs in the group of sedatives, so we save all them in a tuple and search for them in drugs table'


In [165]:
%%time

pharma_records = pd.read_parquet(f's3://icusics-db/pharma_records/pharma_records_h3.parquet')

CPU times: user 1.11 s, sys: 576 ms, total: 1.69 s
Wall time: 1.89 s


In [169]:
pharma_records.head()

,a_patientid,ordernumber,a_pharmaid,time,givendose,routename
0,3001014,804140,3000000005,643,40.0,PERF IV
1,3001014,804140,3000002034,643,50.0,PERF IV
2,3001014,804141,3000000446,233,1.0,PERF IV
3,3001014,804141,3000000446,677,1.0,PERF IV
4,3001014,804141,3000000446,1193,1.0,PERF IV


In [171]:
sed_patlist = tuple(set(pharma_records[(pharma_records['a_patientid'].isin(tuple(set(los2d_pneumopd_imv_apacheII20_lac2fd['a_patientid'])))) & (
    pharma_records['a_pharmaid'].isin(sedatives_ids))]['a_patientid']))

In [175]:
los2d_pneumopd_imv_apacheII20_lac2fd_sedatives = los2d_pneumopd_imv_apacheII20_lac2fd[los2d_pneumopd_imv_apacheII20_lac2fd['a_patientid'].isin(sed_patlist)].sort_values(
    'a_patientid').reset_index(drop=True)

In [177]:
print(los2d_pneumopd_imv_apacheII20_lac2fd_sedatives['a_patientid'].nunique())
print(round(los2d_pneumopd_imv_apacheII20_lac2fd_sedatives['a_patientid'].nunique()/patients['a_patientid'].nunique()*100,2))

26
0.68


In [178]:
print('patients in ICUSICS DB for hospital 3',
      patients['a_patientid'].nunique())
print('from those, with ICU LOS > 2 days:',
      los2d['a_patientid'].nunique())
print('from those, with pneumonia as pd:',
      los2d_pneumopd['a_patientid'].nunique())
print('from those, with imv:',
      los2d_pneumopd_imv['a_patientid'].nunique())
print('from those, with an apache2 > 20:',
      los2d_pneumopd_imv_apacheII20['a_patientid'].nunique())
print('from those, with lactate >2mmol/L at day 1:',
      los2d_pneumopd_imv_apacheII20_lac2fd['a_patientid'].nunique())
print('from those, with sedatives:',
      los2d_pneumopd_imv_apacheII20_lac2fd_sedatives['a_patientid'].nunique())

patients in ICUSICS DB for hospital 3 3815
from those, with ICU LOS > 2 days: 2312
from those, with pneumonia as pd: 228
from those, with imv: 131
from those, with an apache2 > 20: 60
from those, with lactate >2mmol/L at day 1: 26
from those, with sedatives: 26


We see how only 26 patients fullfilled the inclusion criteria of this example. The objective was to explore all tables, not to obtain a real dataset with clinical criteria

In [179]:
los2d_pneumopd_imv_apacheII20_lac2fd_sedatives

,hospital_coded,a_patientid,patientsex,age,height,weight,bmi,hospadmtime,admwardname,distime,diswardname,hospdistime,hospital_outcome
0,3,3006618,M,80,170,90,31,-7221,UROLOGIA,20926,EXITUS,20926,EXITUS
1,3,3010785,M,70,170,100,35,0,H VALLS,67191,EXITUS,67304,EXITUS
2,3,3189352,M,40,180,100,31,0,H VALLS,4516,PNEUMOLOGIA,14757,ALIVE
3,3,3295500,M,30,180,110,34,-1,URGÈNCIES,46245,CIRURGIA ORT I TRAUMATOLOGIA,74523,ALIVE
4,3,3306000,M,60,170,120,42,-5,H VALLS,164080,MEDICINA INTERNA,169641,ALIVE
5,3,3319369,M,70,160,60,23,-10,URGÈNCIES,8083,EXITUS,8093,EXITUS
6,3,3324322,F,60,160,100,39,-247,URGÈNCIES,18387,MEDICINA INTERNA,27041,ALIVE
7,3,3433306,F,60,150,60,27,-12,URGÈNCIES,3892,NEFROLOGIA,32921,ALIVE
8,3,3489340,F,80,150,70,31,-7,URGÈNCIES,21346,HEMATOLOGIA CLÍNICA,58606,ALIVE
9,3,3509270,F,80,160,90,35,-1637,UNITAT CORONARIA,6846,MEDICINA INTERNA,47409,ALIVE


In [180]:
los2d_pneumopd_imv_apacheII20_lac2fd_sedatives['hospital_outcome'].value_counts(normalize=True)

EXITUS    0.653846
ALIVE     0.346154
Name: hospital_outcome, dtype: float64